In [ ]:
import pandas as pd


df=pd.read_csv('reviews1.csv')

under=pd.read_csv('undervalued_true.csv')
id_dict=under.to_dict()
clean_df=df[df['listing_id'].isin(under['id'])].copy()
to_edit=clean_df.groupby(['listing_id'])['comments'].apply(list)
doc=to_edit.iloc[0]

In [ ]:
key = "1769ebac20bf4a02abdad7f27d590df7"
endpoint = "https://tryy.cognitiveservices.azure.com/"


from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()


In [153]:


def sentiment_analysis_example(client, documents, re):

    response = client.analyze_sentiment(documents=documents)
    for r in response:
        to_div=0
        poss=0
        neuu=0
        negg=0
        
        for idx, sentence in enumerate(r.sentences):
            docu =[sentence.text]
            response_lang = client.detect_language(documents = docu, country_hint = 'gr')[0]
            if ( response_lang.primary_language.name=='English'):
                # print("Sentence: {}".format(sentence.text))
                # print("Sentence {} sentiment: {}".format(idx+1, sentence.sentiment))
                # print("Sentence score:\nPositive={0:.2f}\nNeutral={1:.2f}\nNegative={2:.2f}\n".format(
                #     sentence.confidence_scores.positive,
                #     sentence.confidence_scores.neutral,
                #     sentence.confidence_scores.negative))
                
                poss = poss+round(sentence.confidence_scores.positive,2)
                neuu = neuu+round(sentence.confidence_scores.neutral,2)
                negg = negg+round(sentence.confidence_scores.negative,2)
                to_div=to_div+1
        
        if (to_div!=0):
            
            re[0]=re[0]+round(poss/to_div,2)
            re[1]=re[1]+round(neuu/to_div,2)
            re[2]=re[2]+round(negg/to_div,2)
            

        # print('-'*8)
    return re

dict={}
for j in range(0,4):
# len(to_edit)+1):
    doc=to_edit.iloc[j]
    num_rev = np.shape(doc)[0]

    res = [0, 0, 0]
    for i in range(0, num_rev, 10):
        sentiment_analysis_example(client, doc[i:i+10], res)


    x = num_rev % 10
    if x != 0:
        sentiment_analysis_example(client, doc[num_rev-x:num_rev+1], res)

    print(res)
    dict[to_edit.index[j]]=np.divide(res,num_rev)

[16.439999999999998, 2.32, 0.22000000000000003]
[16.65, 2.84, 0.5]
[235.03, 46.64000000000002, 12.339999999999979]
[296.33000000000004, 53.30000000000002, 19.39000000000003]
